In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [8]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [10]:
inspector = inspect(engine)
columns = inspector.get_columns('Measurement')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [ ]:
## Query/retrieve the last 12 months of precip data and plot the results:
## First, Retrieve start and end dates for the period of last 12months: 
# session.query(func.count(Measurement.date)).all() # .first() 
## 19550 rows in Measurements ranging in dates 2010-01-01 to 2017-08-23
## 2016 was a leap year, but 365 days in those last 12months, 2016-08 to 2017-08.
## Capture last date, convert data type to use in timedelta function:
last_day = session.query(Measurement.date).order_by(Measurement.date.desc()).first() 
last_day = str(last_day)
last_date_s= (last_day[2:12]) 
last_date = dt.datetime.strptime(last_date_s, '%Y-%m-%d').date()
# print(type(end_date))    ## <class 'datetime.datetime'>
yr_ago = last_date - dt.timedelta(days=365) 
print(yr_ago)

In [ ]:
# Perform a query to retrieve the data and precipitation scores
YrAgo = session.query(Measurement.station, Measurement.date, \
                      Measurement.prcp).\
                filter(Measurement.date > yr_ago).\
                order_by(Measurement.date).all()

In [ ]:
# Save as a Pandas DataFrame, set index to date column 

yrago_df = pd.DataFrame(YrAgo, columns=['station', 'date', 'precip'])
yrago_df.set_index('date', inplace=True)
yrago_df.head()
# yrago_df.info()       # 2230 rows, 2021 precip values

In [ ]:
# Sort the dataframe by date --- AND GET RID OF NULL (NaN) VALUES ---
yrago_df.sort_values('date', na_position='first')
yrago_df['precip'].isnull().sum().sum()    ## 208 NaNv alues; ALL in precip column
yrago_df.dropna(inplace=True)
yrago_df.head()

In [ ]:
yrago_df.isnull().sum().sum()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
# yrago_df.info()  ## 2021 rows, precip = float
yrago_df.describe()
# yrago_df.info()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
locs = range(0, 2050, 25)
labels = ["2016/08", "2016/09", "2016/010", "2016/011", "2016/012", "2017/01",\
          "2017/02", "2017/03", "2017/04", "2017/05", "2017/06", "2017/07",\
          "2017/08"]
plt.xticks(locs, labels, rotation = 45)
x = yrago_df.index
y = yrago_df["precip"]
plt.xlim(0, 300)
plt.bar(x, y, width=1.5)
plt.tight_layout()
plt.xlabel('Observation dates')
plt.ylabel('Precipitation Score')
plt.title('Daily Rainfall, per Hawaii weather stations')
plt.show()

![precipitation](Images/precipitation.png)

In [ ]:
# Explore Stations table
columns = inspector.get_columns('Station')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Design a query to show how many stations are available in this dataset?
stations = session.query(Station.name).count()
stations

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
StActive = yrago_df.groupby(['station']).size().to_frame('size').\
    reset_index().sort_values(['size'], ascending=[False])
StActive     ## validation: total number of observations is 2015

In [ ]:
sta_active = session.query(Measurement.station, func.count(Measurement.tobs)).\
    group_by(Measurement.station).order_by(func.count(Measurement.tobs).desc()).all()
sta_active    ## validation: total observations = 19550, total rows in Measurement

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
active_min = session.query(func.min(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519397').all()
active_max = session.query(func.max(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519397').all()
active_avg = session.query(func.avg(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519397').all()

active_min, active_max, active_avg


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station...
active = session.query(Measurement.station, Measurement.date, Measurement.tobs).\
    filter(Measurement.station == 'USC00519397').all()
active_sta = pd.DataFrame(active, columns=['station', 'date', 'temp'])
active_sta.describe()

In [ ]:
# ... and plot the results as a histogram with bins = 12
y = active_sta["temp"]
plt.xlim(58, 85)
plt.hist(y, bins=12)
plt.tight_layout()
plt.xlabel('Temperatures')
plt.ylabel('Numbers of times observed')
plt.title('Daily Temperatures, per Hawaii weather stations')

In [ ]:
# Function `calc_temps` will accept start date & end date, format '%Y-%m-%d' 
# and return min, avg, & max temperatures for that range of dates.
def calc_temps(start_date, end_date):
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), \
                         func.max(Measurement.tobs)).\
                    filter(Measurement.date >= start_date).\
                    filter(Measurement.date <= end_date).all()

In [ ]:
# Use the function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
start_date = '2016-09-14'
end_date = '2016-09-29'
templist = calc_temps(start_date, end_date)
templist

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as Title, avg temp for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
x = 22
y = templist[0][1]        ## y.info() shows y value as data type float
yerror = np.array([[(y-templist[0][0]), (templist[0][2]-y)]]).T
yerror

In [ ]:
plt.bar(x, y, width=15, edgecolor="black")
plt.errorbar(x, y, yerr=yerror, color="black")
plt.tight_layout()
plt.ylabel('Temp (F)')
plt.title('Trip Avg Temp')
plt.xlim(1, 45)
plt.xlabel('Days of Trip, September')
plt.show()

In [ ]:
# Calculate total rainfall/ station for trip dates using the previous year for dates.
# Sort in descending order by precipitation amount; list station, name, lat/long, elevation.
start_date = '2016-09-14'
end_date = '2016-09-29'
join_trip = session.query(Measurement.station, Measurement.date, \
                          func.sum(Measurement.prcp), \
                          Station.latitude, Station.longitude, Station.elevation).\
            filter(Measurement.station == Station.station).\
            filter(Measurement.date >= start_date).\
            filter(Measurement.date <= end_date).\
            group_by(Measurement.station).\
            order_by(func.sum(Measurement.prcp).desc()).all()
join_trip

# ********Testing Flask queries********

In [55]:
start_day = '2016-09-14'
start_date = dt.datetime.strptime(start_day, '%Y-%m-%d').date()

end_day = '2016-09-29'
end_date = dt.datetime.strptime(end_day, '%Y-%m-%d').date()
# print(type(start_date))   
# print(start_date, end_date)

num_days = (end_date - start_date).days
# print(num_days)

precip_trip = session.query(Measurement.date, func.sum(Measurement.prcp)).\
                            filter(Measurement.date >= start_day).\
                            filter(Measurement.date <= end_day).\
                            group_by(Measurement.date).\
                            order_by(Measurement.date).all()
# print(type(precip_trip))  ## list
# precip_trip
precip_dict = dict(precip_trip)
precip_dict
precip = list(np.ravel(precip_dict, order='C'))
precip
# precip_days = precip[0:num_days+1]
# precip_prcp = precip[num_days+1:-1]
# print (f"During your trip on {start_day} to {end_day} expect it to rain this much on each of {num_days} days:")
# print(precip_prcp)

# precip_trip_dict = to_dict(precip_trip)
# precip_trip_dict = precip_trip.json()
# for row in precip_trip:
                            
#                             return [dict(row) 

[{'2016-09-14': 14.280000000000001,
  '2016-09-15': 4.96,
  '2016-09-16': 0.8200000000000001,
  '2016-09-17': 0.6799999999999999,
  '2016-09-18': 0.53,
  '2016-09-19': 0.32,
  '2016-09-20': 1.03,
  '2016-09-21': 1.17,
  '2016-09-22': 1.44,
  '2016-09-23': 1.57,
  '2016-09-24': 0.28,
  '2016-09-25': 0.09,
  '2016-09-26': 1.67,
  '2016-09-27': 1.3599999999999999,
  '2016-09-28': 0.13,
  '2016-09-29': 2.99}]

## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (AVGs for tmin, tmax, tavg for all data matching a month+day)
def daily_normals(date):
    """Daily Normals.
    Args:
        date (str): A date string in the format '%m-%d'
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    """
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def mydaynorms(date):
    join_trip = session.query(Measurement.station, Measurement.date, \
                          func.sum(Measurement.prcp), \
                          Station.latitude, Station.longitude, Station.elevation).\
            filter(Measurement.station == Station.station).\
            filter(Measurement.date >= start_date).\
            filter(Measurement.date <= end_date).\
            group_by(Measurement.station).\
            order_by(func.sum(Measurement.prcp).desc()).all()
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
